<a href="https://colab.research.google.com/github/Goodboey/FPLI-MP-SEBASTIAN-EJAZ-EBUBEKIR/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note that lines starting with a # in python are comments meaning either they are explanations to someone trying to read the code, or they are bits of code that we, for now, don't want to use, but may use (by "uncommenting" them) later. In this first section we have statements to import various useful python modules.

In [4]:
import numpy as np
import math

import matplotlib.pyplot as plt
import time
import pandas as pd
#import numba # add @numba.njit before functions to get just-in-time compilation
plt.style.use('classic')

from gravity import gravity

ModuleNotFoundError: ignored

The next cell sets up various units and constants relevant for solar system calculations. Choosing a unit for length, a unit for time, and a unit for mass is enough to define the units for everything else.

In [3]:
G_SI = 6.67408e-11 # Newton's gravitational constant in SI units (m^3 kg^-1 s^-2)

# choose units for mass, length, time. This choice affects the
# numerical value we use for G
M = 1.98855e30 # solar mass in kg
L = 149597870700 # AU in m
T = 365.256*24*3600 # sidereal year in seconds


m_E = 5.972e24 / M # earth-mass in our chosen (solar) mass unit
m_S = 1.98855e30 / M # solar mass: 1 in solar units
m_J = 1.898e27 / M  # Jupiter mass
day = 24*3600 / T  # a day in our chosen time unit, for convenience


# Gravitational constant in chosen units
G = G_SI*M*T**2/L**3
# for solar mass/AU/year the value of G is simply 4 pi^2


## User data: initial positions and velocities
Put the sun initially at the origin. This is where you make changes to do your own simulation. You need to have a position (x,y,z) and a velocity (x,y,z) and a mass for each object. By leaving out the z-components you can do 2-dimensional simulations

In [1]:

positions = np.array([[0., 0., 0.], [?, ?, ? ]])   
velocities = np.array([[0., 0., 0.], [?, ?, ?] ])
masses = np.array([m_S, ?])


SyntaxError: ignored

Various initialization stuff before the main loop starts. There are some lines involving "output_file" which are commented out. These could be used as an alternative 

In [ ]:
nparticles = positions.shape[0]
D = positions.shape[1] ; # number of dimensions

# set the CM velocity to zero. Transpose to get last dimensions to match for
# broadcasting to work
total_momentum = np.sum(masses * velocities.transpose(), 1)
v_CM = total_momentum / np.sum(masses)
velocities -= v_CM

 
dt = 0.001 # time step in years. Can use day as time step by
            # writing dt = 1*day;

T = 10.0 # total time to simulate in years
nsteps = int(T/dt)

write_interval = 10 # write only every 10 (or whatever you choose) time steps to save time and space

# if writing to a file as you go, it needs to be opened now.
#output_file = open('orbits.dat','w')
# header for the output file
#output_file.write("time,xS,yS,xE,yE,d_ES\n")


# technical detail: adjust velocities to get them for a half time step
# backwards (Leap-Frog velocities are defined at the half-way points between
# "real" timesteps).

forces = np.zeros(positions.shape)
gravity(positions, masses, forces, G)
mass_array = np.tile(masses, (3,1)).transpose() # make an Nx3 version of masses
velocities -=  0.5 * (forces / mass_array)  * dt;

## Now down to business: the main simulation loop
This is a function called "run". Something only happens-see below-when that function is called 

In [ ]:
#@numba.njit
def run(positions, velocities, forces, masses, mass_array, G, dt, nsteps, write_interval, output):

   
    for s in range(nsteps):
        # these three lines do all the calculations for the simulation
        gravity(positions, masses, forces, G)
        velocities += (forces / mass_array ) * dt
        positions = positions +  velocities * dt

        # the rest of the main loop is generating output
        if s % write_interval == 0:
            # for plotting the earth-sun distance
            d_ES = math.sqrt(np.sum((positions[1,:]-positions[0,:])**2));

            # IF WRITING TO AN OUTPUT FILE DURING THE SIMULATION
            # the number of items in the format string must match the
            # number of items following
            # you should update the header string above as well
            #output_file.write('%.4f,%.5g,%.5g,%.5g,%.5g,%.5g\n' % (s*dt, positions[0, 0], positions[0, 1], positions[1,0], positions[1,1], d_ES) )

            # If only using pandas dataframe without writing to a file (yet)
            index = s // write_interval # integer division
            output[index, 0] = s*dt # time
            output[index, 1] = positions[0, 0]
            output[index, 2] = positions[0, 1]
            output[index, 3] = positions[1, 0]
            output[index, 4] = positions[1, 1]
            output[index, 5] = d_ES


Here we define the basic output structure: how many quantities (ncols) and how many lines of data. Think of the output as being written to a file with nlines lines and ncols columns.

In [ ]:
nlines = nsteps // write_interval # we want integer division here!
ncols = 6 # how many columns of data (how many different variables)
output_array = np.zeros((nlines, ncols))


In [ ]:
tStart = time.time(); # start timer
run(positions, velocities, forces, masses, mass_array, G, dt, nsteps, write_interval, output_array)
tElapsed = time.time() - tStart # record time taken for main loop
print("Time taken for %d steps is %.3f seconds" % (nsteps, tElapsed))

#output_file.close()

The next section is for plotting the data

In [ ]:
# load data from the file
#df = pd.read_csv('orbits.dat')

# OR construct the DataFrame from the numpy arrays we have
df = pd.DataFrame(output_array, columns=['time','xS','yS','xE','yE','d_ES'])


plt.figure(figsize=(8, 5))
plt.plot(df['time'],df['d_ES'])
plt.title("Simulation of earth orbiting the sun")
plt.xlabel("time")
plt.ylabel("distance of Earth from sun")
    
plt.savefig("Earth-sun-dist-vs-t.png")
plt.show() # makes new fig so call after savefig!

# save the data frame to csv file
df.to_csv('orbits.csv')
